# What is Dingocar

ToDo

# TDX Dingocar Exploration

The aim of this notebook is to implement the Dingocar self-driving roboar CNN using TFX components.

In [ ]:
!pip install "tfx==0.21.2" "tensorflow>=2.1,<2.2" "tensorboard>=2.1,<2.3" "pyzmq==17.0.0"

**Not sure how many of these we need, will remove undeeded later.**

In [ ]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen

from tfx.components import Trainer
from tfx.components import Transform
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

# Download data from public google drive

In [ ]:
%%bash
pip install gdown
mkdir data
gdown --id 1gv5k5vK90QOSgenwT42DMm-jmBdB9yEX --output data/tub.zip
(cd data && unzip tub.zip > _ && cd ..)
echo "Number of examples: `ls data/tub/*.jpg | wc -l`"

## Connect to Google Drive

This piece of code will mount the your google drive to this Google Colab virtual machine. It will prompt you to follow a link to get a verification code. Once you get it, copy and paste it in the box provided and hit enter.

You can nevigate the file system by clicking the "Files" tab in the  <-- left side bar. All your google drive files should be in `/content/drive/My\ Drive`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# MAKE SURE THESE MATCH THE PATH OF THE IMPORTED TUB
from pathlib import Path
_data_root=Path('/content/data/tub/')

_tf_record_dir = Path('/content/data/tf-records')
_tf_record_dir.mkdir(exist_ok=True)

# This is the root directory for your TFX pip package installation.
# _tfx_root = tfx.__path__[0]

# # This is the path where your model will be pushed for serving.
# _serving_model_dir = os.path.join(
#     tempfile.mkdtemp(), 'serving_model/dingo')

# # Set up logging.
# absl.logging.set_verbosity(absl.logging.INFO)

## Import some required modules

In [ ]:
%matplotlib inline
import matplotlib
from matplotlib.pyplot import imshow
import os
from PIL import Image
from glob import glob
import numpy as np
import json
from tqdm import tqdm

## Load and visualise the data

The Dingocar model takes a single image, passes it through a CNN and attempts to output both steering and throttle commands. The data is in the following form.

```
data_dir/
  record_xxx.json
  ...
  xxx_cam-image_array_.jpg
  ...
```

Where `xxx` corrispond to image/label pairs, ie `record_0.json` is the label for the image `0_cam-image_array_.jpg`.

The contents of `record_xxx.json` looks like:

```json
{
  "timestamp": "2019-04-03 07:39:09.646350",
  "cam/image_array": "0_cam-image_array_.jpg",
  "user/mode": "user",
  "user/throttle": 0,
  "user/angle": 0
}
```

Where:

  - `timestamp`: is the time at which the frame was captured
  - `cam/image_array`: the name of the corrisonding image
  - `user/mode`: who was driving the car at the time of data acquisition
  - `user/throttle`: float 0-1 corrisponding to throttle command at the time the frame was captured
  - `user/angle`: float 0-1 corrisponding to steering command at the time the frame was captured.

We're only really interested in `user/angle` and `user/throttle` and the image data for this work.

Here we are simply loading and displaying a single image/label pair to check that things have been downloaded correctly.

In [ ]:
from PIL import Image

label_paths = list(_data_root.glob("record_*.json"))

IMAGE_KEY    = "cam/image_array"
STEERING_KEY = "user/angle"
THROTTLE_KEY = "user/throttle"

def read_image(path):
  img = Image.open(path)
  img = np.array(img, dtype=np.uint8)
  return img

def read_label(path):
  with path.open('r') as f:
    label = json.load(f)
  return label

# Read a single label
idx = 123
label = read_label(label_paths[idx])
image_name = label[IMAGE_KEY]
image_path = str(_data_root / image_name)
image = read_image(image_path)

print(f"Steering: {label[STEERING_KEY]}")
print(f"Throttle: {label[THROTTLE_KEY]}")
imshow(image)

# Convert to TFRecord

The first step in the TFX pipeline is `ExampleGen` this expects our data to be in a specific format `TFRecord`. This is a serialized data format that can be a bit cumbersome to deal with for a small project like this, but it serves two main pourposes.

1. When training models (expecially smaller ones like this) sometimes the a major bottleneck is not the model backprop step, it is simply loading data into the gpu. To run a maximum efficency you want your gpu to be maxed out a much as possiable. Serialized data facilitates this.

2. When training across multiple machines in the cloud, data transfer from storage to the training machine becomes a segnificant bottleneck. Serialized data can be moved between infrastructure faster.

`TFRecords` are composed of `TFExamples`. Each `TFExample` is a image/lable pair
with some additional information to help with decoding the data later.

In [ ]:
# Where to save the resulting TFRecord
tfrecords_filename = str(_tf_record_dir / "tf_record.record")

# The following functions can be used to convert a value to a type compatible
# with tf.Example.
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def read_image_as_byte_string(path):
  # "image/raw" has bytes value "None" which cannot be decoded as a UTF-8 string.
  image_string = open(str(path), 'rb').read()
  return image_string


def image_example(record_path, data_dir):
  """Loads an image/label pair and converts to a tf.Example

  Args:
    record_path (str): Path to a record.json
    data_dir (str): Path to the directory where records and images are stored

  Returns:
    tf.Example
  """
  label = read_label(record_path)
  steering = label[STEERING_KEY]
  throttle = label[THROTTLE_KEY]
  image_name = label[IMAGE_KEY]
  image_path = _data_root / image_name
  image_string = read_image_as_byte_string(image_path)
  image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'image/height': _int64_feature(image_shape[0]),
      'image/width': _int64_feature(image_shape[1]),
      'image/depth': _int64_feature(image_shape[2]),
      'label/steering': _float_feature(steering),
      'label/throttle': _float_feature(throttle),
      'image/raw': _bytes_feature(image_string),
  }
  return tf.train.Example(features=tf.train.Features(feature=feature))

with tf.io.TFRecordWriter(tfrecords_filename) as writer:
  for path in label_paths:
    example = image_example(path, _data_root)
    writer.write(example.SerializeToString())


# Read TFRecord

This step is not crucial, we're just reading in a the `TFRecord` and decoding a `TFExample` to see what it looks like

In [ ]:
def decode_tfexample(example):
  """Decode a single tf.Example

  Args:
    example (tf.Example): A single tf.Example 

  Returns:
    image/label pair (np.array, (float, float)): (image array, (steering, throttle))

  """

  height = (example.features.feature['image/height']
                                .int64_list
                                .value)[0]
  
  width = (example.features.feature['image/width']
                              .int64_list
                              .value)[0]

  img_string = (example.features.feature['image/raw']
                                .bytes_list
                                .value)[0]
  
  steering = (example.features.feature['label/steering']
                            .float_list
                            .value)[0]

  throttle = (example.features.feature['label/throttle']
                        .float_list
                        .value)[0]

  img_flat = tf.image.decode_jpeg(img_string).numpy()
  image_arr = img_flat.reshape((120, 160, -1))

  return image_arr, (steering, throttle)

filenames = [tfrecords_filename]
raw_dataset = tf.data.TFRecordDataset(filenames)

for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  (image_arr, (steering, throttle)) = decode_tfexample(example)
print(f'label: {steering}')
print(f'throttle: {throttle}')
imshow(image_arr)
  

In [ ]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

# Example Gen

In [ ]:
from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.import_example_gen.component import ImportExampleGen
from  tfx.proto import example_gen_pb2

examples = external_input(_tf_record_dir)
# https://www.tensorflow.org/tfx/guide/examplegen#custom_inputoutput_split
# has a good explanation of splitting the data the 'output_config' param

# Input train split is _tf_record_dir/*'
# Output 2 splits: train:eval=8:2.

train_ratio = 8
eval_ratio  = 10-train_ratio
output = example_gen_pb2.Output(
             split_config=example_gen_pb2.SplitConfig(splits=[
                 example_gen_pb2.SplitConfig.Split(name='train',
                                                   hash_buckets=train_ratio),
                 example_gen_pb2.SplitConfig.Split(name='eval',
                                                   hash_buckets=eval_ratio)
             ]))
example_gen = ImportExampleGen(input=examples,
                               output_config=output)

In [ ]:
context.run(example_gen)

# Decode Example From Example Gen. Just to see if it worked.

In [ ]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

In [ ]:
# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 1 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

(image_arr, (steering, throttle)) = decode_tfexample(example)
print(f'label: {steering}')
print(f'throttle: {throttle}')
imshow(image_arr)

# Stastics Gen

In [ ]:
  # stats_options = StatsOptions(
  #     enable_semantic_domain_stats = True)

# Squashing logging so StatisticsGen doesn't complain
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
statistics_gen = StatisticsGen(
  examples=example_gen.outputs['examples'])

###   ☹ ☹ ☹ ☹ WARNINGS OCCUR HERE ☹ ☹ ☹ ☹

In [ ]:
context.run(statistics_gen)

In [ ]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

In [ ]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

In [ ]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

In [ ]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

In [ ]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

# Old Donkey car code below. I think  I need this for the Transfrom step

## Data Augmentation

Data augmentation allows us to add a bit more variety to the training data. One very handy augmentation transformation is to randomly mirror the input image and the steering label. This ensurse the data contains the same number or left and right turns so the neural network does not become bias to a specifc direction of turn. 

There are also some other augmentation transformations you can apply below. These will hopefully make the network a bit more robust to canging lighting and help prevent overfitting.

In [ ]:
import config
from functools import partial
from dingocar.parts.augmentation import apply_aug_config

# Play with the data augmentation settings if you like
# In all cases 'aug_prob' is the probability the given 
# augmentation will occure. All the other parameters are
# explained below.
aug_config = {

# Mirror the image horizontally
"mirror_y"         : {"aug_prob" : 0.5},

# Randomly turn pixels black of white.
# "noise" : The probability a pixel is affected.
#           0.0 : No pixels will be effected
#           1.0 : All pixels will be effected
                 "salt_and_pepper"  : {"aug_prob" : 0.3,
                                       "noise"    : 0.2},

# Randomly turn pixels a random color
# "noise" : The probability a pixel is affected.
#           0.0 : No pixels will be effected
#           1.0 : All pixels will be effected
                 "100s_and_1000s"   : {"aug_prob" : 0.3,
                                       "noise"    : 0.2},

# Randomly increase or decrease the pixel values by an 
# value between 'min_val' and 'max_val'. The resulting
# value will be clipped between 0 and 255    
                 "pixel_saturation" : {"aug_prob" : 0.3,
                                       "min_val"  :-20,
                                       "max_val"  : 20},

# Randomly shuffle the RGB channel order
                 "shuffle_channels" : {"aug_prob" : 0.3},

# Randomly set a rectangular setction of the image to 0
# the rectangle height and width is randomly generated 
# to be between dimention*min_frac and dimention*max_frac.
# So min_frac = 0.0 and max_frac = 1.0 would result
# in a random rectangel that could cover the entire image, 
# or none of the image or anywhere inbetween.
                 "blockout"         : {"aug_prob" : 0.3,
                                       "min_frac" : 0.07,
                                       "max_frac" : 0.3}
                }

# If you're unfamiliar with the 'partial' function. It allows you to
# call a function with some of the arguments pre-filled.
# In this case we made a function that is like 'apply_aug_config', but
# has the `aug_config` parameter pre-filled.
record_transform=partial(apply_aug_config, aug_config=aug_config)

record = tub.get_record(idx, record_transform=record_transform)
print(f"Steering: {record[STEERING_KEY]}")
print(f"Throttle: {record[THROTTLE_KEY]}")
imshow(record[IMAGE_KEY])

## Define the CNN

In [ ]:
from tensorflow.python.keras.layers import Convolution2D
from tensorflow.python.keras.layers import Dropout, Flatten, Dense
from dingocar.parts.keras import KerasLinear
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model, load_model

# Tub objects maintain a dictionary of data. You can access the data via 'keys'.
# Traditionally x stands for inputs and y stands for outputs.
# In our case, for every input image (x) there are 2 output labels,
# steering angle and throttle (y).
X_KEYS = [IMAGE_KEY]
Y_KEYS = [STEERING_KEY, THROTTLE_KEY]

# If you'd like you can play with this neural network as much as you like. See
# if you can get the network to be more accurate!
# The only things you need to watch out for are:
#   1. 'img_in' cannot change.
#   2. 'angle_out' must always haev 'units=1'
#   3. 'throttle_out' must always have 'units=1'
def convolutional_neural_network():
    img_in = Input(shape=(120, 160, 3), name='img_in')                                                                                                                       
    x = img_in                                                                                                                                                               
    
    # Convolution2D class name is an alias for Conv2D 
    x = Convolution2D(filters=24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)                                                                                  
    x = Convolution2D(filters=32, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)                                                                                  
    x = Convolution2D(filters=64, kernel_size=(5, 5), strides=(2, 2), activation='relu')(x)                                                                                  
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(2, 2), activation='relu')(x)                                                                                  
    x = Convolution2D(filters=64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(x)                                                                                  
    
    x = Flatten(name='flattened')(x)
    x = Dense(units=100, activation='linear')(x)                                                                                                                             
    x = Dropout(rate=.2)(x)
    x = Dense(units=50, activation='linear')(x)                                                                                                                              
    x = Dropout(rate=.2)(x)
    # categorical output of the angle
    angle_out = Dense(units=1, activation='linear', name='angle_out')(x)                                                                                                     
    
    # continous output of throttle
    throttle_out = Dense(units=1, activation='linear', name='throttle_out')(x)                                                                                               
    
    model = Model(inputs=[img_in], outputs=[angle_out, throttle_out])                                                                                                        
    
    model.compile(optimizer='adam',
                  loss={'angle_out': 'mean_squared_error',
                        'throttle_out': 'mean_squared_error'},
                  loss_weights={'angle_out': 0.5, 'throttle_out': 0.5})                                                                                                       
    
    return model

# KerasLinear is a class the contains some functions we can use to train
# our model and to get predictions out if it later.
model = KerasLinear(model=convolutional_neural_network())

## Train the model


In [ ]:
from manage import train
import config

# Load 16 image at a time into the model
BATCH_SIZE       = 32

# 70% of the data is used for training. 30% for validation
TRAIN_TEST_SPLIT = 0.7

# Number of time to look over all the training data
EPOCHS = 100

# Stop training if the validation loss has not improved for the last 'PATIENTS'
# Epochs.
USE_EARLY_STOP = True 
PATIENCE = 5


# Where to save the trained model
new_model_path = "/content/drive/My Drive/dingocar/no_mirror1.hdf5"

# If you want to start from a pre-trained model you can add the path here
base_model_path   = None


# These are generators that will be used to feed data into the model
# when training. The generator uses a constant random seed so the train/val
# split is the same every time.
train_gen, val_gen = tub.get_train_val_gen(X_KEYS, Y_KEYS,
                      batch_size=BATCH_SIZE,
                      train_frac=TRAIN_TEST_SPLIT,
                      train_record_transform=record_transform,
                      val_record_transform=None)

training_history = model.train(train_gen,
                               val_gen,
                               new_model_path,
                               epochs=EPOCHS,
                               patience=PATIENCE,
                               use_early_stop=USE_EARLY_STOP)

# Visualize Predictions

In [ ]:
from dingocar.parts.keras import KerasLinear

new_model_path = "/content/drive/My Drive/dingocar/no_mirror1.hdf5"
trained_model = new_model_path

# Load a pre-trained model
model = KerasLinear()
model.load(trained_model)


In [ ]:
from dingocar.parts.datastore import Tub

_, val_gen = tub.get_train_val_gen(X_KEYS, Y_KEYS,
                      batch_size=1,
                      train_frac=TRAIN_TEST_SPLIT,
                      train_record_transform=None,
                      val_record_transform=None)

In [ ]:
preds = []
truth = []

val_count = int(tub.get_num_records() * (1-TRAIN_TEST_SPLIT))
for _ in tqdm(range(val_count)):
    sample = next(val_gen)
    pred = model.run(sample[0][0][0])
    preds.append(pred)
    truth.append((sample[1][0][0], sample[1][1][0]))
        
preds = np.array(preds)
truth = np.array(truth)
print(preds.shape)
print(truth.shape)


In [ ]:
import matplotlib.pyplot as plt

def mean_squared_error(preds, true):
  squared_error = (true - preds)**2
  return np.mean(squared_error)

def xy_scatter(preds, truth):
    fig = plt.figure(figsize=(14,14))
    steering_p = preds[...,0]
    throttle_p = preds[...,1]
    steering_t = truth[...,0]
    throttle_t = truth[...,1]
    
    steering_mse = mean_squared_error(steering_p, steering_t)
    throttle_mse = mean_squared_error(throttle_p, throttle_t)
    plt.plot(steering_p, steering_t, 'b.')
    plt.title(f"MSE: {steering_mse:.3f}")
    plt.xlabel("predictions")
    plt.ylabel("ground truth")
    plt.gca().set_xlim(-1, 1)
    plt.show()
#     fig = plt.gcf()
#     fig.savefig(path + "/pred_vs_anno.png", dpi=100)

# Only display the validation set
xy_scatter(preds, truth)

In [ ]:
preds = []
truth = []

for idx in tqdm(range(tub.get_num_records())):
    sample = tub.get_record(idx)
    pred = model.run(sample[IMAGE_KEY])
    preds.append(pred)
    truth.append((sample[STEERING_KEY], sample[THROTTLE_KEY]))
        
preds = np.array(preds)
truth = np.array(truth)
print(preds.shape) 


In [ ]:
from ipywidgets import interact, fixed
import ipywidgets as widgets

In [ ]:

def plt_image(ax, image, title):
  ax.imshow(image)
  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_title(title)

def plt_samples(idxs, axs, tub):
  records = [tub.get_record(i) for i in idxs]
  images = [r[IMAGE_KEY] for r in records]
  titles = [f"frame: {i}" for i in idxs]
  for a,i,t in zip(axs, images, titles):
    plt_image(a,i,t)
        
def time_series(x=300):#, axs=axs, tub=tub):
  
    fig = plt.figure(figsize=(21,12))
    plt.tight_layout()

    ax1 = plt.subplot2grid((2, 5), (0, 0), colspan=5)
    ax2 = plt.subplot2grid((2, 5), (1, 0))
    ax3 = plt.subplot2grid((2, 5), (1, 1))
    ax4 = plt.subplot2grid((2, 5), (1, 2))
    ax5 = plt.subplot2grid((2, 5), (1, 3))
    ax6 = plt.subplot2grid((2, 5), (1, 4))
    axs = [ax2, ax3, ax4, ax5, ax6]
    steering_p = preds[...,0]
    throttle_p = preds[...,1]
    steering_t = truth[...,0]
    throttle_t = truth[...,1]
  
    idxs = np.arange(x-2,x+3)
    plt_samples(idxs, axs, tub)
  
    start = x-300
    end  = x + 300
    ax1.plot(steering_p, label="predictions")
    ax1.plot(steering_t, label="ground truth")
    #ax1.axvline(x=x, linewidth=4, color='r')
    ax1.legend(bbox_to_anchor=(0.91, 0.96), loc=2, borderaxespad=0.)
    ax1.set_title("Time Series Throttle Predictions vs Ground Truth")
    ax1.set_xlabel("time (frames)")
    ax1.set_ylabel("steering command")
    ax1.set_xlim(start, end)
    
#time_series(x=600)
interact(time_series, x=(300, len(truth-300)))#, axs=fixed(axs), tub=fixed(tub))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt    
testData = np.array([[0,0], [0.1, 0], [0, 0.3], [-0.4, 0], [0, -0.5]])
fig, ax = plt.subplots()
sctPlot, = ax.plot(testData[:,0], testData[:,1], "o", picker = 5)
plt.grid(True)
plt.axis([-0.5, 0.5, -0.5, 0.5])

def on_pick(event):
    artist = event.artist
    artist.set_color(np.random.random(3))
    print("click!")
    fig.canvas.draw()

fig.canvas.mpl_connect('pick_event', on_pick)